# 2024 COMP90042 Project
*Make sure you change the file name with your group id.*

# Readme
*If there is something to be noted for the marker, please mention here.*

*If you are planning to implement a program with Object Oriented Programming style, please put those the bottom of this ipynb file*

# 0.Setting Colab Method for future model developing
Firstly, run the following block to mount the drive to the colab. Then, drag the data folder/**eval.py** to the "Colab Folder Space" to ensure the code runs successfully.

If data folder updated, attempt to forcibly remount, call `drive.mount("/content/drive", force_remount=True)`.


In [1]:
#from google.colab import drive
#drive.mount('/content/drive')

# 1.DataSet Processing
(You can add as many code blocks and text blocks as you need. However, YOU SHOULD NOT MODIFY the section title)

In [2]:
import json
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
import operator
from statistics import mean
from collections import Counter

## 1.1 Reading and gathering data

In [3]:
with open('data/train-claims.json', 'r') as input_file:
    train_claim_data = json.load(input_file)

# Read in development data (claim)
with open('data/dev-claims.json', 'r') as input_file:
    dev_claim_data = json.load(input_file)

# Read in test data (claim)
with open('data/test-claims-unlabelled.json', 'r') as input_file:
    test_claim_data = json.load(input_file)

# Read in evidence data
with open('data/evidence.json', 'r') as input_file:
    evi_data = json.load(input_file)

#EDA


claim_count = 0
evi_count = 0
claim_length = []
evidence_count = []
evidence_length = []
labels = []

for key,value in train_claim_data.items():
    claim_count+=1
    claim_length.append(len(value["claim_text"]))
    evidence_count.append(len(value["evidences"]))
    evidence_length += [len(evi_data[x]) for x in value["evidences"]]
    labels.append(value["claim_label"])

for key,value in evi_data.items():
    evi_count+=1

print("claim count: ",claim_count)
print("evidence count: ",evi_count)
print("max claim length: ",max(claim_length))
print("min claim length: ",min(claim_length))
print("mean claim length: ",mean(claim_length))
print("max evidence count: ",max(evidence_count))
print("min evidence count: ",min(evidence_count))
print("mean evidence count: ",mean(evidence_count))
print("max evidence length: ",max(evidence_length))
print("min evidence length: ",min(evidence_length))
print("mean evidence length: ",mean(evidence_length))
print(Counter(labels))



inside = 0
outside = 0

train_evi_id = []
for claim_id,claim_value in train_claim_data.items():
    train_evi_id=train_evi_id+claim_value['evidences']

for claim_id,claim_value in dev_claim_data.items():
    test_evi_id=claim_value['evidences']
    for e in test_evi_id:
        if e in train_evi_id:
            inside += 1
        else:
            outside += 1
print("Dev evi inside train evi", inside)
print("Dev evi outside train evi", outside)

full_evidence_id = list(evi_data.keys())
full_evidence_text  = list(evi_data.values())
train_claim_id = list(train_claim_data.keys())
train_claim_text  = [ v["claim_text"] for v in train_claim_data.values()]


claim count:  1228
evidence count:  1208827
max claim length:  332
min claim length:  26
mean claim length:  122.95521172638436
max evidence count:  5
min evidence count:  1
mean evidence count:  3.3566775244299674
max evidence length:  1979
min evidence length:  13
mean evidence length:  173.5
Counter({'SUPPORTS': 519, 'NOT_ENOUGH_INFO': 386, 'REFUTES': 199, 'DISPUTED': 124})
Dev evi inside train evi 163
Dev evi outside train evi 328


## 1.2 Remove unnecessary data

In [4]:
import nltk
nltk.download('words')

[nltk_data] Downloading package words to
[nltk_data]     C:\Users\ABC\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


True

In [5]:
print(full_evidence_id[0:10])

['evidence-0', 'evidence-1', 'evidence-2', 'evidence-3', 'evidence-4', 'evidence-5', 'evidence-6', 'evidence-7', 'evidence-8', 'evidence-9']


In [6]:
import string

# remove non-english words
def is_pure_english(text):
    english_letters = set(string.ascii_letters)
    cleaned_text = ''.join(char for char in text if char.isalpha() or char.isspace())
    return all(char in english_letters or char.isspace() for char in cleaned_text)

In [7]:
print(len(set(full_evidence_text)))
print(len(full_evidence_text))
print(next(iter(evi_data.items())))

1193821
1208827
('evidence-0', 'John Bennet Lawes, English entrepreneur and agricultural scientist')


In [8]:
# find the keys of a dictionary given a set of values
def remove_non_eng(dictionary):
    
    # create a dictionary with values as keys and keys as values
    eng_evi = {}

    # populate the dictionary
    for key, value in dictionary.items():
        if is_pure_english(value):
            eng_evi[key] = value
    
    return eng_evi

# find the keys of the english evidence
eng_evi = remove_non_eng(evi_data)
print(len(eng_evi))
print(len(evi_data))

1114577
1208827


In [9]:
english_evidence = list(eng_evi.values())
english_evidence_id = list(eng_evi.keys())

In [10]:
import re  # 引入正则表达式

# 权威网站 https://www.ucdavis.edu/climate/definitions
climate_keywords = [
    "climate", "environment", "global warming", "greenhouse effect", "carbon", "co2", "carbon dioxide", 
    "methane", "renewable energy", "sustainability", "ecology", "biodiversity", 
    "fossil fuels", "emissions", "air quality", "ozone", "solar energy", "wind energy", 
    "climate change", "climate crisis", "climate adaptation", "climate mitigation", 
    "ocean", "sea levels", "ice melting", "deforestation", "reforestation", "pollution"
]


# 支持模糊匹配的函数
def contains_climate_keywords(text, keywords):
    text = text.lower()  # 将文本转为小写以忽略大小写差异
    for keyword in keywords:
        if re.search(r"\b" + re.escape(keyword) + r"\b", text):  # 使用正则表达式匹配整个词
            return True
    return False

# 移除与气候科学无关的项
def filter_climate_related(dictionary, keywords):
    cs_evi = {}
    for key, value in dictionary.items():
        if contains_climate_keywords(value, keywords):
            cs_evi[key] = value
    return cs_evi


In [11]:
cs_evi = filter_climate_related(eng_evi, climate_keywords)

In [12]:
print(len(eng_evi))
print(len(cs_evi))

1114577
19324


In [18]:
climate_evidence = list(cs_evi.values())
climate_evidence_id = list(cs_evi.keys())

# 2. Model Implementation
(You can add as many code blocks and text blocks as you need. However, YOU SHOULD NOT MODIFY the section title)

In [19]:

# dev remove stop word会好一点， test不remove反而好
evidence_tfidf_vectorizer = TfidfVectorizer(stop_words="english", use_idf=True)

# claim_tfidf_vectorizer = TfidfVectorizer(stop_words="english", use_idf=True)
claim_tfidf_vectorizer = TfidfVectorizer(use_idf=True)


evidence_tfidf_vectorizer.fit(train_claim_text+climate_evidence) #english_evidence#
train_claim_emb_list = claim_tfidf_vectorizer.fit_transform(train_claim_text)


full_evi_emb_list = evidence_tfidf_vectorizer.transform(climate_evidence) #english_evidence#


In [20]:
print(full_evi_emb_list.shape)
print(full_evi_emb_list[0])

(19324, 25597)
  (0, 21985)	0.31907363046479503
  (0, 21280)	0.3184043764287972
  (0, 18471)	0.26539378264997104
  (0, 18148)	0.21522476161607026
  (0, 17980)	0.1961832613321785
  (0, 10265)	0.2796006490876323
  (0, 8346)	0.3121812193340289
  (0, 8224)	0.2553347461544191
  (0, 7545)	0.42354351380620653
  (0, 4554)	0.4704770601718685


In [21]:
evi_k=2
claim_k=1

with open('data/dev-claims.json', 'r') as input_file:
    test_out_temp = json.load(input_file)

for claim_id,claim_value in test_out_temp.items():
    # Task1
    # 把test claim转化成vector

    test_claim_emb = evidence_tfidf_vectorizer.transform([claim_value['claim_text']])
    evi_sim_dict = {}
    # 计算出test claim和所有evidence的相似度
    sim = cosine_similarity(test_claim_emb, full_evi_emb_list)[0]

    for i in range(len(sim)):
        evi_sim_dict[climate_evidence_id[i]] = sim[i] #full_evidence_id[i]

    # 对evidence根据和claim的相似度排序
    s_sim = [(k, v) for k, v in sorted(evi_sim_dict.items(), key=lambda item: item[1],reverse=True)][:evi_k]
    sel_sim = [k for k,v in s_sim]
    # 把最相似的前k个evidence的id写入到test claim的evidence list
    test_out_temp[claim_id]["evidences"] = sel_sim

    # Task2
    # 把test claim转化成vector 注意我这两问用了不同的vectorizer，因为有不同的预处理步骤，和应用数据目标
    test_claim_emb = claim_tfidf_vectorizer.transform([claim_value['claim_text']])

    # 计算出test claim和所有train claim的相似度
    claim_sim_dict = {}
    claim_sim = cosine_similarity(test_claim_emb, train_claim_emb_list)[0]
    for i in range(len(claim_sim)):
        claim_sim_dict[train_claim_id[i]] = claim_sim[i]

    # 取最相似的k个train claim
    most_sim_claims = [(k, v) for k, v in sorted(claim_sim_dict.items(), key=lambda item: item[1],reverse=True)]
    # 我这里用的是k=1只考虑最相似的那一个，label拿出来
    most_sim_claim = max(most_sim_claims, key=operator.itemgetter(1))[0]

    test_out_temp[claim_id]["claim_label"] = train_claim_data[most_sim_claim]["claim_label"]



# Writing to sample.json
with open("data/dev_predict.json", "w") as outfile:
    json.dump(test_out_temp, outfile)

In [25]:

print(len(full_evidence_id))
print(len(english_evidence_id))
print(full_evi_emb_list.shape)
#没写进去 可能是sim没算出来#
#(1114577, 479945)
#0.0 但是text用english但id用full就会是0.1057.....

1208827
1114577
(19324, 25597)


# 3.Testing and Evaluation
(You can add as many code blocks and text blocks as you need. However, YOU SHOULD NOT MODIFY the section title)

In [24]:
# %%cmd
# python eval.py --predictions dev-claims-baseline.json --groundtruth dev-claims.json
# python eval.py --predictions dev_predict.json --groundtruth dev-claims.json


import subprocess

# proc = subprocess.Popen(["python", "eval.py", "--predictions", "data\dev_predict.json", "--groundtruth", "data\dev-claims.json"
# ], stdout=subprocess.PIPE, shell=True)
# (out, err) = proc.communicate()
# print(str(out))

# 高自动化模型/预处理选择，可以自动读取准确度
output = subprocess.check_output("python eval.py --predictions data/dev_predict.json --groundtruth data/dev-claims.json", shell=True)
output_str = output.decode('utf-8')

# Split the output into lines
output_lines = output_str.strip().split('\n')

# Format the output
formatted_lines = []
for line in output_lines:
    metric, value = line.split('=')
    metric = metric.strip()
    value = value.strip()
    formatted_line = f"{metric}: {value}"
    formatted_lines.append(formatted_line)

# Join the formatted lines into a single string
formatted_output = '\n'.join(formatted_lines)
print(formatted_output)

Evidence Retrieval F-score (F): 0.07272727272727272
Claim Classification Accuracy (A): 0.474025974025974
Harmonic Mean of F and A: 0.1261066724249622


## Object Oriented Programming codes here

*You can use multiple code snippets. Just add more if needed*